# Rigorous Neuro-Symbolic Reasoning

This notebook demonstrates a hybrid architecture that addresses key challenges in combining machine learning with formal logic:

1. **Symbol Grounding Problem**: How do we map natural language to logical forms?
2. **Epistemic vs. Validity**: Separating argument structure from premise truth
3. **Confidence Calculus**: Explicit propagation rules
4. **Fallacy Detection**: Independent validation of LLM reasoning
5. **Unparseable Fragments**: What can't be formalized?

## Architecture

```
ML Backend (Claude + Extended Thinking)
           ↓
    Integration Layer
           ↓
Logic Frontend (Formal Proofs + KB)
```

In [ ]:
# Setup
import sys
from pathlib import Path

# Add agents directory
agents_dir = Path.cwd()
if agents_dir.name != 'agents':
    agents_dir = agents_dir / 'agents'
sys.path.insert(0, str(agents_dir))

print(f"✅ Working directory: {Path.cwd()}")

## 1. Symbol Grounding Problem

**Challenge**: How does "All ML models trained on biased data produce biased outputs" become `∀x(BiasedTraining(x) → BiasedOutput(x))`?

**Answer**: Context-dependent semantic interpretation with explicit grounding rules.

In [ ]:
from logic.grounding import SemanticParser, create_ml_context

# Create ML-specific semantic context
ml_context = create_ml_context()

print("📚 ML Semantic Context")
print(f"\nPredicates:")
for pred, definition in ml_context.predicates.items():
    print(f"  • {pred}: {definition}")

# Parse with context
parser = SemanticParser(ml_context)

statement = "All ML models trained on biased data produce biased outputs"
result = parser.parse(statement)

print(f"\n📝 Parsing: '{statement}'")
print(f"\nSuccess: {result.success}")
print(f"Logical Form: {result.logical_form}")
print(f"Confidence: {result.confidence:.1%}")
print(f"\nAssumptions:")
for assumption in result.assumptions:
    print(f"  • {assumption}")

## 2. Quantification: "All" vs. "Most" vs. "Some" vs. Generic

**Challenge**: These have different logical structures and some can't be expressed in first-order logic.

In [ ]:
test_cases = [
    ("All swans are white", "Universal: ∀x(Swan(x) → White(x))"),
    ("Some swans are white", "Existential: ∃x(Swan(x) ∧ White(x))"),
    ("Swans are white", "Generic: Gen x(Swan(x) → White(x))"),
    ("Most swans are white", "❌ UNPARSEABLE in first-order logic")
]

print("🔍 QUANTIFICATION ANALYSIS\n")
print("=" * 70)

for statement, expected in test_cases:
    result = parser.parse(statement)
    
    print(f"\n'{statement}'")
    print(f"  Expected: {expected}")
    
    if result.success:
        print(f"  ✅ {result.logical_form}")
        print(f"  Confidence: {result.confidence:.1%}")
        if result.assumptions:
            print(f"  ⚠️  {result.assumptions[0]}")
    else:
        print(f"  ❌ Cannot formalize: {result.unparseable_fragments[0]}")

## 3. Epistemic Status vs. Logical Validity

**Key Distinction**:
- **Validity**: Conclusion follows from premises (structural correctness)
- **Soundness**: Premises are actually true (factual correctness)

An argument can be valid but unsound (Socrates is a fish).

In [ ]:
from logic.epistemic import ValidityChecker

checker = ValidityChecker()

# Example: Valid but absurd
print("🔹 CASE 1: Valid but Absurd\n")
print("Premise 1: If Socrates is a fish, then Socrates can swim")
print("Premise 2: Socrates is a fish")
print("Conclusion: Socrates can swim")

status = checker.check_modus_ponens(
    "If Socrates is a fish, then Socrates can swim",
    "Socrates is a fish",
    "Socrates can swim"
)

print(f"\nValidity: {status.validity.value} (confidence: {status.validity_confidence:.1%})")
print(f"Soundness: {status.soundness.value}")
print(f"Justification: {status.validity_justification}")
print(f"\n💡 This argument is VALID but UNSOUND")
print(f"   The structure is correct, but premise 2 is false.")

## 4. Fallacy Detection

**Challenge**: LLMs might label reasoning as "modus ponens" when it's actually "affirming the consequent" (a fallacy).

**Solution**: Independent validation that doesn't trust the LLM's self-labeling.

In [ ]:
from logic.epistemic import ConfidenceCalculator

calc = ConfidenceCalculator()

print("🔍 FALLACY DETECTION\n")
print("=" * 70)

# Example 1: Affirming the consequent
print("\n📋 Example 1: Missing Universal Quantifier")
premises = ["Tech companies are successful", "Google is a tech company"]
conclusion = "Google is successful"
claimed_rule = "modus_ponens"

print(f"Premises: {premises}")
print(f"Conclusion: {conclusion}")
print(f"Claimed Rule: {claimed_rule}")

fallacy = calc.detect_fallacy(premises, conclusion, claimed_rule)

if fallacy:
    print(f"\n🚨 FALLACY DETECTED: {fallacy}")
    print(f"\nProblem: Premise says 'Tech companies are successful' (generic/existential)")
    print(f"         Valid form requires: 'ALL tech companies are successful'")
    print(f"         This is affirming the consequent, not modus ponens.")
else:
    print("\n✅ No fallacy detected")

# Example 2: Hasty generalization
print("\n" + "=" * 70)
print("\n📋 Example 2: Hasty Generalization")
premises2 = ["Some swans I've seen are white"]
conclusion2 = "All swans are white"
claimed_rule2 = "universal_generalization"

print(f"Premises: {premises2}")
print(f"Conclusion: {conclusion2}")
print(f"Claimed Rule: {claimed_rule2}")

fallacy2 = calc.detect_fallacy(premises2, conclusion2, claimed_rule2)

if fallacy2:
    print(f"\n🚨 FALLACY DETECTED: {fallacy2}")
    print(f"\nProblem: Generalizing from 'some' to 'all' without sufficient sample.")
else:
    print("\n✅ No fallacy detected")

## 5. Explicit Confidence Propagation

**Challenge**: If I chain 10 steps each at 0.9 confidence, what's the final confidence?

**Answer**: Explicit calculus with multiple methods:
- Conjunctive (AND): Product rule
- Disjunctive (OR): Complement rule
- Inference steps: Weighted by rule type

In [ ]:
print("📊 CONFIDENCE PROPAGATION\n")
print("=" * 70)

# Show degradation with chain length
print("\n📉 Conjunctive Chain Degradation:")
print("(Each step at 0.9 confidence)\n")

for length in [1, 3, 5, 10, 20]:
    confidences = [0.9] * length
    result = calc.conjunctive_chain(confidences)
    print(f"  {length:2d} steps → {result:.1%}")

print("\n💡 Observation: Confidence degrades exponentially")
print("   10 steps at 0.9 → only 35% confidence!")
print("   This is why shorter proofs are preferred.")

# Show multi-step breakdown
print("\n" + "=" * 70)
print("\n📋 Multi-Step Reasoning Example:\n")
print("P1: All ML models trained on biased data produce biased outputs (0.7)")
print("P2: GPT-4 was trained on biased data (0.6)")
print("R1: Universal instantiation (0.9)")
print("C: GPT-4 produces biased outputs (?)")

breakdown = calc.multi_step_chain(
    premise_confidences=[0.7, 0.6],
    rule_confidences=[0.9],
    rule_types=["universal_instantiation"]
)

print(f"\n🔬 Confidence Breakdown:")
print(f"  Logical Confidence: {breakdown.logical_confidence:.1%}")
print(f"    (How confident are we the argument is valid?)")
print(f"  Source Confidence: {breakdown.source_confidence:.1%}")
print(f"    (How confident are we the premises are true?)")
print(f"  Propagation Method: {breakdown.propagation_method}")
print(f"  Chain Length: {breakdown.chain_length}")

## 6. Modality Detection

**Challenge**: "Most birds can fly" and "John believes Mary is happy" can't be expressed in first-order logic.

**Solution**: Detect when specialized logics are needed and report unparseable fragments.

In [ ]:
print("🔍 MODALITY DETECTION\n")
print("=" * 70)

modality_cases = [
    ("All humans are mortal", "First-order logic"),
    ("Most birds can fly", "Requires higher-order logic"),
    ("John believes Mary is happy", "Requires epistemic logic"),
    ("It's illegal to drive without insurance", "Requires deontic logic"),
    ("The water boiled because it was heated", "Requires causal logic")
]

for statement, logic_needed in modality_cases:
    result = parser.parse(statement)
    
    print(f"\n'{statement}'")
    print(f"  Required: {logic_needed}")
    
    if result.modality:
        print(f"  Modality: {result.modality.value}")
    
    if result.success:
        print(f"  ✅ {result.logical_form}")
    else:
        print(f"  ❌ {result.unparseable_fragments[0]}")

## 7. Full Reasoning Agent Integration

Putting it all together: A reasoning agent that:
1. Parses with context-aware grounding
2. Validates argument structure independently
3. Propagates confidence explicitly
4. Detects fallacies
5. Reports unparseable fragments

In [ ]:
from logic import ReasoningAgent, LogicType

# Create agent with first-order logic
agent = ReasoningAgent(
    name="Rigorous Reasoner",
    system_prompt="Combines ML reasoning with formal logic validation",
    logic_framework=LogicType.FIRST_ORDER,
    reasoning_depth=3,
    logic_weight=0.75,  # Prioritize logic over consensus
    verbose=False
)

# Add knowledge with provenance
print("📚 Building Knowledge Base...\n")

agent.add_knowledge(
    "All software engineers write code",
    source="domain_knowledge",
    confidence=1.0
)

agent.add_knowledge(
    "Alice is a software engineer",
    source="user_input",
    confidence=1.0
)

# Reason
print("❓ Query: What can we conclude about Alice?\n")

result = agent.reason("What can we conclude about Alice?")

# Display results
print("=" * 70)
print("📊 REASONING RESULT")
print("=" * 70)

print(f"\nConclusion: {result['conclusion']}")
print(f"Formal: {result['formal_conclusion']}")
print(f"Confidence: {result['confidence']:.1%}")

print(f"\n🔗 Reasoning Chain:")
for i, step in enumerate(result['reasoning_chain'], 1):
    print(f"  {i}. {step['premise']}")
    print(f"     → [{step['rule']}]")
    print(f"     → {step['conclusion']}")
    print(f"     (confidence: {step['confidence']:.1%})")

print(f"\n✅ Knowledge Validation:")
val = result['knowledge_validation']
print(f"  Valid: {val['valid']}")
print(f"  KB Confidence: {val['confidence']:.1%}")
print(f"  Sources: {', '.join(val['sources'])}")

# Show formal argument
print("\n" + "=" * 70)
print("📜 FORMAL ARGUMENT")
print("=" * 70)

formatter = agent.argument_builder.formatter
print(formatter.format_argument(result['formal_argument']))

## Summary

This neuro-symbolic system addresses key challenges:

### ✅ Symbol Grounding
- Context-dependent semantic interpretation
- Explicit predicate definitions
- Grounding rules for term mapping

### ✅ Epistemic vs. Validity
- Separate tracking of structural validity and factual truth
- Valid arguments can have false premises
- Invalid arguments can have true premises

### ✅ Confidence Calculus
- Explicit propagation rules (product, complement)
- Multi-dimensional confidence (logical, empirical, source)
- Chain length effects are visible and justified

### ✅ Fallacy Detection
- Independent validation (doesn't trust LLM self-labeling)
- Detects affirming consequent, hasty generalization, etc.
- Checks for missing quantifiers

### ✅ Unparseable Fragments
- Gracefully handles what can't be formalized
- Reports which specialized logic is needed
- Makes assumptions explicit

## Next Steps

For production systems:
1. Full NLP parser (dependency trees, semantic roles)
2. Automated theorem proving (Coq, Lean, Isabelle)
3. Higher-order logic for "most", "few"
4. Modal logics for beliefs, time, causation
5. Ground truth oracles (external validation)
6. Contradiction detection and resolution